# Installing and importing modules

In [ ]:
try:
  !pip install annoy
  print('Installed Annoy')
except:
  print('error')          

     |████████████████████████████████| 655kB 22.0MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391603 sha256=17a7093be3140428bf11388e172fb9f3b342a7a6cf0763878c81b78675b9224c
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy
Installed Annoy


In [ ]:
from annoy import AnnoyIndex
from scipy import spatial
import tensorflow_hub as hub
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import glob
import pickle
import re

from skimage.io import imread
from sklearn.metrics.pairwise import cosine_similarity
import scipy as sc

import io
import sys
import os
import time   

from PIL import Image
from numpy import asarray
import pandas as pd

# Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Pickle files/image_paths','rb') as f:
  image_paths = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/Pickle files/final_dataframe','rb') as f:
  df = pickle.load(f)

In [ ]:
#df.reset_index(drop=False,inplace=True)
df

,gender,category,sub-category,product-number,product-name,path
0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...
8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...
...,...,...,...,...,...,...
201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg
201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg


In [ ]:
im_ar = asarray(Image.open(final_path[0]))
im_ar.shape

(417, 152, 3)

In [ ]:
image_paths[0][0]

'/content/drive/MyDrive/women/tops/long_sleeved_tops/89728708/89728708_1.jpeg'

In [ ]:
final_path = {}
for x in df.itertuples():
  final_path[x.Index] = '/content/drive/MyDrive/'+ x.path

In [ ]:
path_in_drive = pd.DataFrame(pd.Series(final_path))
path_in_drive

,0
0,/content/drive/MyDrive/women/dresses/casual_an...
3,/content/drive/MyDrive/women/dresses/casual_an...
4,/content/drive/MyDrive/women/dresses/casual_an...
6,/content/drive/MyDrive/women/dresses/casual_an...
8,/content/drive/MyDrive/women/dresses/casual_an...
...,...
201831,/content/drive/MyDrive/women/tops/t-shirts/914...
201832,/content/drive/MyDrive/women/tops/t-shirts/914...
201833,/content/drive/MyDrive/women/tops/t-shirts/914...
201834,/content/drive/MyDrive/women/tops/t-shirts/914...


In [ ]:
df = pd.concat([df,path_in_drive],axis=1)
df.rename(columns={0:'path_in_drive'},inplace=True)
df

,gender,category,sub-category,product-number,product-name,path,path_in_drive
0,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,/content/drive/MyDrive/women/dresses/casual_an...
3,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,/content/drive/MyDrive/women/dresses/casual_an...
4,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,/content/drive/MyDrive/women/dresses/casual_an...
6,women,dresses,casual_and_day_dresses,56037632,black knit midi dress,women/dresses/casual_and_day_dresses/56037632/...,/content/drive/MyDrive/women/dresses/casual_an...
8,women,dresses,casual_and_day_dresses,62629380,multicolor henley t-shirt dress,women/dresses/casual_and_day_dresses/62629380/...,/content/drive/MyDrive/women/dresses/casual_an...
...,...,...,...,...,...,...,...
201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...


In [ ]:
df[df['path_in_drive'] == '/content/drive/MyDrive/women/skirts/maxi_skirts/90311765/90311765_3.jpeg']

,gender,category,sub-category,product-number,product-name,path,path_in_drive
142967,women,skirts,maxi_skirts,90311765,multicolor liv elephant print double split max...,women/skirts/maxi_skirts/90311765/90311765_3.jpeg,/content/drive/MyDrive/women/skirts/maxi_skirt...


In [ ]:
df.drop(index=142964,axis=0,inplace=True)

In [ ]:
files = glob.glob('/content/drive/MyDrive/npz files /*.npz')

In [ ]:
def sort_glob(file):
  file_path_dic ={}
  for f in file:
    file_path_dic[int(re.findall('[0-9]+',f)[0])] = f

  file_paths = []
  sorted_file_path = sorted(file_path_dic.items())

  for tup in sorted_file_path:
    file_paths.append(tup[1])
  
  return file_paths

In [ ]:
files = sort_glob(files)
index = [int(x.split('/')[-1][:-4]) for x in files]
len(files)

91465

In [ ]:
new_df = df[~df.index.isin(index)]
new_df

,gender,category,sub-category,product-number,product-name,path,path_in_drive
200351,women,tops,t-shirts,91272357,multicolor print t-shirt,women/tops/t-shirts/91272357/91272357_2.jpeg,/content/drive/MyDrive/women/tops/t-shirts/912...
200353,women,tops,t-shirts,91272357,multicolor print t-shirt,women/tops/t-shirts/91272357/91272357_4.jpeg,/content/drive/MyDrive/women/tops/t-shirts/912...
200355,women,tops,t-shirts,91272661,purple ribbed cotton tee,women/tops/t-shirts/91272661/91272661_0.jpeg,/content/drive/MyDrive/women/tops/t-shirts/912...
200357,women,tops,t-shirts,91272661,purple ribbed cotton tee,women/tops/t-shirts/91272661/91272661_2.jpeg,/content/drive/MyDrive/women/tops/t-shirts/912...
200362,women,tops,t-shirts,91273108,black crystal embellished t-shirt,women/tops/t-shirts/91273108/91273108_2.jpeg,/content/drive/MyDrive/women/tops/t-shirts/912...
...,...,...,...,...,...,...,...
201831,women,tops,t-shirts,91437517,classic tee black,women/tops/t-shirts/91437517/91437517_2.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201832,women,tops,t-shirts,91437722,black red heart t-shirt,women/tops/t-shirts/91437722/91437722_0.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201833,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_0.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...
201834,women,tops,t-shirts,91437753,white run shirt,women/tops/t-shirts/91437753/91437753_1.jpeg,/content/drive/MyDrive/women/tops/t-shirts/914...


# Coverting image into a vector

In [ ]:
def load_img(path):

  im_ar = asarray(Image.open(path))
  img = im_ar
  img = tf.image.resize_with_pad(img,224,224)
  img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis,...]
  return img

def get_image_feature_vectors():
  i = 0
  j = 1
  module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4"
  module = hub.load(module_handle)
  
  for pi_id,filename in zip(new_df.index,new_df['path_in_drive']):
    if i == 50:
      print(j)
      j+=1
      i = 0
    i +=1
    #print("image count           :%s"%i)

    img = load_img(filename)
    features = module(img)
    #print(features)
    feature_set = np.squeeze(features)
    #print(feature_set)
    #print(feature_set.shape)
    np.savez('/content/drive/MyDrive/npz files /'+str(pi_id)+'.npz',feature_set)


In [ ]:
get_image_feature_vectors() 

1
2
3
4
5
6
7
8
9
10
11
12
13


In [ ]:
i = np.load('/content/drive/MyDrive/npz files /3.npz')
i['arr_0']

array([2.3490744 , 0.9729177 , 0.        , ..., 0.        , 0.        ,
       0.01716174], dtype=float32)